In [1]:
import pandas as pd
import numpy as np
import json
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import requests
import re

In [2]:
# Windows users
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 94.0.4606
[WDM] - Get LATEST driver version for 94.0.4606


[WDM] - Get LATEST driver version for 94.0.4606
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/94.0.4606.61/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\ajgli\.wdm\drivers\chromedriver\win32\94.0.4606.61]


In [78]:
lahman_player_csv = "core\People.csv"
lahman_team_csv = "core\Teams.csv"
lahman_batting_csv = "core\Batting.csv"

In [80]:
lahman_player_df = pd.read_csv(lahman_player_csv)
lahman_team_df = pd.read_csv(lahman_team_csv)
lahman_batting_df = pd.read_csv(lahman_batting_csv)
lahman_player_df

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,...,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
0,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,Aardsma,David Allan,215.0,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01
1,aaronha01,1934.0,2.0,5.0,USA,AL,Mobile,2021.0,1.0,22.0,...,Aaron,Henry Louis,180.0,72.0,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01
2,aaronto01,1939.0,8.0,5.0,USA,AL,Mobile,1984.0,8.0,16.0,...,Aaron,Tommie Lee,190.0,75.0,R,R,1962-04-10,1971-09-26,aarot101,aaronto01
3,aasedo01,1954.0,9.0,8.0,USA,CA,Orange,NaN,NaN,NaN,...,Aase,Donald William,190.0,75.0,R,R,1977-07-26,1990-10-03,aased001,aasedo01
4,abadan01,1972.0,8.0,25.0,USA,FL,Palm Beach,NaN,NaN,NaN,...,Abad,Fausto Andres,184.0,73.0,L,L,2001-09-10,2006-04-13,abada001,abadan01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20353,zupofr01,1939.0,8.0,29.0,USA,CA,San Francisco,2005.0,3.0,25.0,...,Zupo,Frank Joseph,182.0,71.0,L,R,1957-07-01,1961-05-09,zupof101,zupofr01
20354,zuvelpa01,1958.0,10.0,31.0,USA,CA,San Mateo,NaN,NaN,NaN,...,Zuvella,Paul,173.0,72.0,R,R,1982-09-04,1991-05-02,zuvep001,zuvelpa01
20355,zuverge01,1924.0,8.0,20.0,USA,MI,Holland,2014.0,9.0,8.0,...,Zuverink,George,195.0,76.0,R,R,1951-04-21,1959-06-15,zuveg101,zuverge01
20356,zwilldu01,1888.0,11.0,2.0,USA,MO,St. Louis,1978.0,3.0,27.0,...,Zwilling,Edward Harrison,160.0,66.0,L,L,1910-08-14,1916-07-12,zwild101,zwilldu01


In [225]:
lahman_player_df['Full Name'] = lahman_player_df['nameFirst'] + " " + lahman_player_df["nameLast"]

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,...,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID,Full Name
7124,griffke01,1950.0,4.0,10.0,USA,PA,Donora,NaN,NaN,NaN,...,George Kenneth,190.0,71.0,L,L,1973-08-25,1991-05-31,grifk001,griffke01,Ken Griffey
7125,griffke02,1969.0,11.0,21.0,USA,PA,Donora,NaN,NaN,NaN,...,George Kenneth,195.0,75.0,L,L,1989-04-03,2010-05-31,grifk002,griffke02,Ken Griffey


In [3]:
mvp_url = 'https://www.baseball-reference.com/awards/mvp.shtml'
war_url = 'https://www.espn.com/mlb/war/leaders/_/type/seasonal/year/'

In [4]:
df = pd.read_html(mvp_url)

In [5]:
mvp_list = pd.DataFrame(df[0])

In [6]:
mvp_year = np.array(mvp_list[( 'Unnamed: 0_level_0', 'Year')].values)
mvp_name = np.array(mvp_list[( 'Unnamed: 2_level_0', 'Name')].values)
mvp_team = np.array(mvp_list[( 'Unnamed: 3_level_0', 'Tm')].values)
mvp_WAR =  np.array(mvp_list[( 'Unnamed: 4_level_0', 'WAR')].values)

In [7]:
data = {'Year Won': mvp_year, 'Name': mvp_name, 'Team': mvp_team, 'WAR': mvp_WAR}

In [240]:
mvp_df = pd.DataFrame(data)
mvp_df = mvp_df.dropna()

In [241]:
mvp_df['Year Won'] = mvp_df['Year Won'].astype(str)
mvp_df['Year Won']= mvp_df['Year Won'].str[:4]
mvp_df['Year Won'] = mvp_df['Year Won'].astype(int)

In [242]:
next_year = []
for year in mvp_df['Year Won']:
    next_year.append(year + 1)
mvp_df['Next Year'] = next_year
mvp_df

,Year Won,Name,Team,WAR,Next Year
0,2020,José Abreu,CHW,3.0,2021
1,2020,Freddie Freeman,ATL,3.3,2021
3,2019,Mike Trout,LAA,7.9,2020
4,2019,Cody Bellinger,LAD,8.6,2020
6,2018,Mookie Betts,BOS,10.7,2019
...,...,...,...,...,...
296,1913,Jake Daubert,BRO,4.0,1914
298,1912,Tris Speaker,BOS,10.1,1913
299,1912,Larry Doyle,NYG,5.0,1913
301,1911,Ty Cobb,DET,10.7,1912


In [69]:
new_df = mvp_df.loc[mvp_df['Year Won']>=2000]

In [73]:
# next_war = []
# year_array = new_df['Year Won'].unique()
# for row in new_df.index:
#     i = 51
#     year_loop = year_array[row]
#     name = mvp_df['Name'][row]
#     year_url = f'{war_url}{year}'
#     war = ''
#     print(f'{name} {year}')
#     while i <= 201:
#     #Can be while true?
#         year_html = pd.read_html(year_url)
#         year_df = pd.DataFrame(year_html[0])
#         if year_df.loc[year_df[1] == name][2].empty and i != 151:
#             year_url = f'https://www.espn.com/mlb/war/leaders/_/year/{year}/type/seasonal/alltime/false/count/{i}'
#             i= i + 50
#             print(f'{year} {i}')
#         elif year_df.loc[year_df[1] == name][2].empty and i == 151:
#             next_war.append(war)
#             print(f'{year} {year}')
#             break
#         else:
#             war = year_df.loc[year_df[1] == name][2].values[0]
#             next_war.append(war)
#             break
    
    
    

In [74]:
# next_war = []
# current_war = []
# years = new_df['Year Won'].unique()
# for year in years:
#     next_players = new_df.loc[new_df['Next Year'] == year]['Name']
#     for player in next_players:
#         print(f'{player} {year}')
#         i = 51
#         year_url = f'{war_url}{year}'
#         war = ''
#         while i <= 201:
#         #Can be while true?
#             year_html = pd.read_html(year_url)
#             year_df = pd.DataFrame(year_html[0])
#             if year_df.loc[year_df[1] == player][2].empty and i != 151:
#                 year_url = f'https://www.espn.com/mlb/war/leaders/_/year/{year}/type/seasonal/alltime/false/count/{i}'
#                 i= i + 50
#                 print(i)
#             elif year_df.loc[year_df[1] == player][2].empty and i == 151:
#                 next_war.append(war)
#                 print(i)
#                 break
#             else:
#                 war = year_df.loc[year_df[1] == player][2].values[0]
#                 print(war)
#                 next_war.append(war)
#                 break
#     current_players = new_df.loc[new_df['Year Won'] == year]['Name']
#     for player in current_players:
#         print(f'{player} {year}')
#         i = 51
#         year_url = f'{war_url}{year}'
#         war = ''
#         while i <= 201:
#         #Can be while true?
#             year_html = pd.read_html(year_url)
#             year_df = pd.DataFrame(year_html[0])
#             if year_df.loc[year_df[1] == player][2].empty and i != 151:
#                 year_url = f'https://www.espn.com/mlb/war/leaders/_/year/{year}/type/seasonal/alltime/false/count/{i}'
#                 i= i + 50
#             elif year_df.loc[year_df[1] == player][2].empty and i == 151:
#                 current_war.append(war)
#                 break
#             else:
#                 war = year_df.loc[year_df[1] == player][2].values[0]
#                 current_war.append(war)
#                 break
    
    
    

Mike Trout 2020
101
1.6
Cody Bellinger 2020
101
1.4
José Abreu 2020
Freddie Freeman 2020
Mookie Betts 2019
6.8
Christian Yelich 2019
7.1
Mike Trout 2019
Cody Bellinger 2019
Jose Altuve 2018
5.2
Giancarlo Stanton 2018
101
4
Mookie Betts 2018
Christian Yelich 2018
Mike Trout 2017
6.7
Kris Bryant 2017
6.1
Jose Altuve 2017
Giancarlo Stanton 2017
Josh Donaldson 2016
7.4
Bryce Harper 2016
101
151
151
Mike Trout 2016
Kris Bryant 2016
Mike Trout 2015
9.4
Clayton Kershaw 2015
7.6
Josh Donaldson 2015
Bryce Harper 2015
Miguel Cabrera 2014
4.9
Andrew McCutchen 2014
6.4
Mike Trout 2014
Clayton Kershaw 2014
Miguel Cabrera 2013
7.2
Buster Posey 2013
5.2
Miguel Cabrera 2013
Andrew McCutchen 2013
Justin Verlander 2012
7.7
Ryan Braun 2012
7
Miguel Cabrera 2012
Buster Posey 2012
Josh Hamilton 2011
101
3.6
Joey Votto 2011
6.4
Justin Verlander 2011
Ryan Braun 2011
Joe Mauer 2010
5.7
Albert Pujols 2010
7.5
Josh Hamilton 2010
Joey Votto 2010
Dustin Pedroia 2009
5.6
Albert Pujols 2009
9.7
Joe Mauer 2009
Alber

In [71]:
next_war

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 array(['2.9'], dtype=object),
 array(['2.9'], dtype=object),
 '',
 '',
 array([], dtype=object),
 array([], dtype=object),
 '',
 '',
 '',
 '',
 array(['3.5'], dtype=object),
 '',
 '',
 '']

In [61]:
new_df['Next War'] = next_war
year_test = new_df['Year Won'].unique()
new_df.loc[new_df['Next Year'] == year_test[2]]

C:\Users\ajgli\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Year Won,Name,Team,WAR,Next Year,Next War
0,2020,José Abreu,CHW,3.0,2021,
1,2020,Freddie Freeman,ATL,3.3,2021,4.5
3,2019,Mike Trout,LAA,7.9,2020,1.6
4,2019,Cody Bellinger,LAD,8.6,2020,1.4
6,2018,Mookie Betts,BOS,10.7,2019,6.8
7,2018,Christian Yelich,MIL,7.3,2019,7.1
9,2017,Jose Altuve,HOU,7.7,2018,5.2
10,2017,Giancarlo Stanton,MIA,7.9,2018,4
12,2016,Mike Trout,LAA,10.5,2017,6.7
13,2016,Kris Bryant,CHC,7.3,2017,6.1


In [83]:
#This is my testing area
# i = 51
# year = 2020
# year_url = f'{war_url}{year}'
# while i <= 201:
#     year_html = pd.read_html(year_url)
#     year_df = pd.DataFrame(year_html[0])
#     if year_df.loc[year_df[1] == 'Spencer Turnbull'][2].empty:
#         year_url = f'https://www.espn.com/mlb/war/leaders/_/year/{year}/type/seasonal/alltime/false/count/{i}'
#         i= i + 50
#         print('no')
#     else:
#         print(year_df.loc[year_df[1] == 'Spencer Turnbull'][2].values[0])
#         break   
        

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,...,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
0,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,Aardsma,David Allan,215.0,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01
1,aaronha01,1934.0,2.0,5.0,USA,AL,Mobile,2021.0,1.0,22.0,...,Aaron,Henry Louis,180.0,72.0,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01
2,aaronto01,1939.0,8.0,5.0,USA,AL,Mobile,1984.0,8.0,16.0,...,Aaron,Tommie Lee,190.0,75.0,R,R,1962-04-10,1971-09-26,aarot101,aaronto01
3,aasedo01,1954.0,9.0,8.0,USA,CA,Orange,NaN,NaN,NaN,...,Aase,Donald William,190.0,75.0,R,R,1977-07-26,1990-10-03,aased001,aasedo01
4,abadan01,1972.0,8.0,25.0,USA,FL,Palm Beach,NaN,NaN,NaN,...,Abad,Fausto Andres,184.0,73.0,L,L,2001-09-10,2006-04-13,abada001,abadan01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20353,zupofr01,1939.0,8.0,29.0,USA,CA,San Francisco,2005.0,3.0,25.0,...,Zupo,Frank Joseph,182.0,71.0,L,R,1957-07-01,1961-05-09,zupof101,zupofr01
20354,zuvelpa01,1958.0,10.0,31.0,USA,CA,San Mateo,NaN,NaN,NaN,...,Zuvella,Paul,173.0,72.0,R,R,1982-09-04,1991-05-02,zuvep001,zuvelpa01
20355,zuverge01,1924.0,8.0,20.0,USA,MI,Holland,2014.0,9.0,8.0,...,Zuverink,George,195.0,76.0,R,R,1951-04-21,1959-06-15,zuveg101,zuverge01
20356,zwilldu01,1888.0,11.0,2.0,USA,MO,St. Louis,1978.0,3.0,27.0,...,Zwilling,Edward Harrison,160.0,66.0,L,L,1910-08-14,1916-07-12,zwild101,zwilldu01


In [97]:
lahman_player_df.loc[lahman_player_df['Full Name'] == "Juan Soto"]['playerID']

17321    sotoju01
Name: playerID, dtype: object

In [131]:
new_df

,Year Won,Name,Team,WAR,Next Year
0,2020,José Abreu,CHW,3.0,2021
1,2020,Freddie Freeman,ATL,3.3,2021
3,2019,Mike Trout,LAA,7.9,2020
4,2019,Cody Bellinger,LAD,8.6,2020
6,2018,Mookie Betts,BOS,10.7,2019
7,2018,Christian Yelich,MIL,7.3,2019
9,2017,Jose Altuve,HOU,7.7,2018
10,2017,Giancarlo Stanton,MIA,7.9,2018
12,2016,Mike Trout,LAA,10.5,2017
13,2016,Kris Bryant,CHC,7.3,2017


In [250]:
mvp_df.loc[mvp_df['Name'].str[-3:] == 'Jr.', 'Name'] = mvp_df['Name'].str[:-4]
merge_player_df = mvp_df.merge(lahman_player_df, left_on='Name', right_on='Full Name', how='left')
merge_team_df = merge_player_df.merge(lahman_batting_df, left_on=['playerID', 'Next Year'], right_on = ['playerID', 'yearID'], how='left')




In [254]:
merge_cteam_df = merge_player_df.merge(lahman_batting_df, left_on=['playerID', 'Year Won'], right_on = ['playerID', 'yearID'], how='left')
old_team = merge_cteam_df['teamID']
merge_team_df['Team'] = old_team
change_list = [0 if merge_team_df['Team'][i] == merge_team_df['teamID'][i] else 1 for i in merge_team_df.index]
merge_team_df['Changed Team'] = change_list
change_df = merge_team_df.loc[merge_team_df['Changed Team'] == 1][['Name', 'playerID', 'Team', 'teamID', 'Changed Team', 'Full Name']]
changed_df = change_df.dropna()
changed_df = changed_df.drop([33])
changed_df

,Name,playerID,Team,teamID,Changed Team,Full Name
7,Giancarlo Stanton,stantmi03,MIA,NYA,1,Giancarlo Stanton
36,Alex Rodriguez,rodrial01,TEX,NYA,1,Alex Rodriguez
62,Barry Bonds,bondsba01,PIT,SFN,1,Barry Bonds
208,Eddie Collins,collied01,PHA,CHA,1,Eddie Collins
